In [1]:
import pandas as pd
import os

In [2]:
# Create a list to contain a reference name for each csv
file_names = []
# Create a list to contain the path for each csv, this will be used to read the csvs
file_paths = []
# Create a dictionary to contain the dataframes that will be created for each csv
dfs_dict = {}

# Specify the path to search (where your csv downloads exist)
path = r"/Data_folder"

# Create a file counter (gets plugged into the reference name of the dataframe)
f = 0

# Walk through the path provided and build a list of all files with ".csv" suffix
for root, dirs, files in os.walk(path):
    for name in files:
        if ".csv" in name:
            # Commit the file name to a list
            file_names.append("csv_{}".format(f))
            # Commit the file path to a list (for reading later)
            file_paths.append(os.path.join(path, name))
            f += 1
        else:
            pass

# For each csv found within the folder read the csv into a dataframe
for n, csv in zip(file_names, file_paths):
    # Read csv and specify some new field names
    dfs_dict[n] = pd.read_csv(csv, sep=",", header=None, names=["timestamp", "river_flow_M3_S"])
    # There"s a big header block in each file. Since we're collating many sensors we'll transpose
    # some of the header info into useful fields.    
    dfs_dict[n]["station_name"] = dfs_dict[n].iloc[1]["river_flow_M3_S"]
    dfs_dict[n]["station_lat"] = dfs_dict[n].iloc[2]["river_flow_M3_S"]
    dfs_dict[n]["station_long"] = dfs_dict[n].iloc[3]["river_flow_M3_S"]
    dfs_dict[n]["station_no"] = dfs_dict[n].iloc[8]["river_flow_M3_S"]
    dfs_dict[n]["station_id"] = dfs_dict[n].iloc[9]["river_flow_M3_S"]
    # All done transposing the header so we'll drop those rows from our dataframe
    dfs_dict[n] = dfs_dict[n].drop([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11])

    # Re-order the columns we created
    column_names = ["station_no", "station_id", "station_name", "timestamp", "river_flow_M3_S",
                    "station_lat", "station_long"]
    dfs_dict[n] = dfs_dict[n].reindex(columns = column_names)

# For all the files we gathered and formatted append them to one dataframe
df = pd.concat([v for k, v in dfs_dict.items()], ignore_index=True, sort=False)

In [3]:
df.head(10)

,station_no,station_id,station_name,timestamp,river_flow_M3_S,station_lat,station_long
0,85,14827,York,2006-01-01T00:00:00,103.29999999999997,43.021702537,-79.89126664
1,85,14827,York,2006-01-01T01:00:00,103.0,43.021702537,-79.89126664
2,85,14827,York,2006-01-01T02:00:00,103.29999999999997,43.021702537,-79.89126664
3,85,14827,York,2006-01-01T03:00:00,103.29999999999997,43.021702537,-79.89126664
4,85,14827,York,2006-01-01T04:00:00,103.29999999999997,43.021702537,-79.89126664
5,85,14827,York,2006-01-01T05:00:00,103.29999999999997,43.021702537,-79.89126664
6,85,14827,York,2006-01-01T06:00:00,103.29999999999997,43.021702537,-79.89126664
7,85,14827,York,2006-01-01T07:00:00,103.29999999999997,43.021702537,-79.89126664
8,85,14827,York,2006-01-01T08:00:00,103.29999999999997,43.021702537,-79.89126664
9,85,14827,York,2006-01-01T09:00:00,103.0,43.021702537,-79.89126664


In [4]:
# Now that all of our data is assembled lets format the columns
# Format the column datatypes
df["timestamp"] = pd.to_datetime(arg = df["timestamp"], format="%Y-%m-%dT%H:%M:%S")
df["river_flow_M3_S"] = pd.to_numeric(arg = df["river_flow_M3_S"], errors = "coerce")
df["station_lat"] = pd.to_numeric(arg = df["station_lat"], errors = "coerce")
df["station_long"] = pd.to_numeric(arg = df["station_long"], errors = "coerce")

# Sort the dataset by the timestamp field so it's in logical order
df = df.sort_values(by="timestamp")

In [5]:
df.head(10)

,station_no,station_id,station_name,timestamp,river_flow_M3_S,station_lat,station_long
0,85,14827,York,2006-01-01 00:00:00,103.300000,43.021703,-79.891267
424592,144,14575,Aberfoyle,2006-01-01 00:00:00,0.468358,43.454656,-80.162625
82682,91,15056,Leggatt,2006-01-01 00:00:00,7.113855,43.967280,-80.354884
486443,93,15051,Keldon,2006-01-01 00:00:00,3.237210,44.070820,-80.370170
901102,16,15157,Elmira,2006-01-01 00:00:00,0.962400,43.601811,-80.556390
626951,33,15012,Clair Creek,2006-01-01 00:00:00,0.433875,43.461917,-80.541856
348342,184,14983,Victoria Road,2006-01-01 00:00:00,5.742258,43.587903,-80.274031
458870,81,14434,Upper Belwood,2006-01-01 00:00:00,32.308403,43.829211,-80.298853
1,85,14827,York,2006-01-01 01:00:00,103.000000,43.021703,-79.891267
82683,91,15056,Leggatt,2006-01-01 01:00:00,7.038529,43.967280,-80.354884


In [ ]:
# This looks much better, lets bring our monitoring data into Insights for further analysis
%insights_return(df)